# Testing grounds

In [1]:
import kwant
import numpy as np 
from numpy import linalg as LA
import scipy.sparse.linalg as sla
from types import SimpleNamespace
import holoviews as hv
from holoviews import opts
from matplotlib import pyplot as plt
hv.extension('matplotlib', logo=False) #BOKEH DOESNT SUPPORT LATEX IN LABELS
hv.output(fig='svg')
import time

s_0 = np.identity(2)
s_z = np.array([[1, 0], [0, -1]])
s_x = np.array([[0, 1], [1, 0]])
s_y = np.array([[0, -1j], [1j, 0]])
# pauli = [sx,sy,sz] #pauli = {'x':sx, 'y':sy, 'z':sz} #as dictionary

#define the Pauli matrices 
pauli = SimpleNamespace(s0=np.array([[1., 0.], [0., 1.]]),
                        sx=np.array([[0., 1.], [1., 0.]]),
                        sy=np.array([[0., -1j], [1j, 0.]]),
                        sz=np.array([[1., 0.], [0., -1.]]))

pauli.s0s0 = np.kron(pauli.s0, pauli.s0) # I(4)
pauli.s0sx = np.kron(pauli.s0, pauli.sx) # 4x4 \sigma_x
pauli.s0sy = np.kron(pauli.s0, pauli.sy) # 4x4 \sigma_y
pauli.s0sz = np.kron(pauli.s0, pauli.sz) # 4x4 \sigma_z
pauli.sxs0 = np.kron(pauli.sx, pauli.s0) # \tau_x
pauli.sys0 = np.kron(pauli.sy, pauli.s0) # \tau_y
pauli.szs0 = np.kron(pauli.sz, pauli.s0) # \tau_z

C:\ProgramData\Anaconda3\envs\lps\lib\site-packages\kwant\solvers\default.py:18: RuntimeWarning: MUMPS is not available, SciPy built-in solver will be used as a fallback. Performance can be very poor in this case.
  "Performance can be very poor in this case.", RuntimeWarning)


In [47]:
#define a Boolean function to shape your system
radius = 15
def disk(position): 
    x,y = position
    return x**2 + y**2 < radius**2

def magn_texture(position,azi_winding, radi_winding):
    x,y = position
    theta = np.arctan2(x,y)
    q = azi_winding
    p = radi_winding
    R = radius
    r = np.sqrt(x**2 + y**2)
    B = [np.sin(np.pi*p*(r/R))*np.cos(q*theta), np.sin(np.pi*p*(r/R))*np.sin(q*theta), np.cos(np.pi*p*(r/R))]
    return B

def onsite(site, t, mu, j, azi_winding, radi_winding, delta): #define a function to determine the onsite energy term of the Hamiltonian
    position = site.pos #site is a class! Apart from real space position contains the type of atom (to which family it belongs, how many orbitals etc)
    B = magn_texture(position,azi_winding,radi_winding) #calculate direction of magnetic field at position (x,y)
    skyrmion_interaction = j*(B[0]*pauli.s0sx + B[1]*pauli.s0sy + B[2]*pauli.s0sz)
    return 4*t*pauli.szs0 - mu*pauli.szs0 + pauli.szs0*mu + delta*pauli.sxs0
    
def hopping(position1,position2,t): #define the hopping terms in your system
    return -t*pauli.szs0

In [48]:
sys = kwant.Builder() #initialize your system
sqlat = kwant.lattice.square()


sys[sqlat.shape(disk,(0,0))]= onsite
sys[sqlat.neighbors()]= hopping

# kwant.plot(sys)
sys= sys.finalized()

In [49]:
pars = dict(t=1, mu=-0.1, j=0.2, delta=0.2, azi_winding=1, radi_winding=1)
ham = sys.hamiltonian_submatrix(params=pars) #is np a variable when calling this function instead of pacakage numpy?

In [50]:
ham

array([[ 4. +0.j,  0. +0.j,  0.2+0.j, ...,  0. +0.j,  0. +0.j,  0. +0.j],
       [ 0. +0.j,  4. +0.j,  0. +0.j, ...,  0. +0.j,  0. +0.j,  0. +0.j],
       [ 0.2+0.j,  0. +0.j, -4. +0.j, ...,  0. +0.j,  0. +0.j,  0. +0.j],
       ...,
       [ 0. +0.j,  0. +0.j,  0. +0.j, ...,  4. +0.j,  0. +0.j,  0.2+0.j],
       [ 0. +0.j,  0. +0.j,  0. +0.j, ...,  0. +0.j, -4. +0.j,  0. +0.j],
       [ 0. +0.j,  0. +0.j,  0. +0.j, ...,  0.2+0.j,  0. +0.j, -4. +0.j]])

In [51]:
t1 = time.time()
eVal,eVec =LA.eigh(ham)
t2 = time.time()
print(t2-t1)

8.308781385421753


In [ ]:
t1 = time.time()
eVal, eVec = sla.eigsh(ham_sparse,k=6, which = 'SM')
t2 = time.time()

t2-t1

In [26]:
np.shape(ham)

(2788, 2788)

In [52]:
hv.Path((np.arange(len(eVal)),eVal))

:Path   [x,y]

In [10]:
len(sys.onsites)

697